### AutoScript Client

In [ ]:
%load_ext autoreload
%autoreload 2
from odemis.driver.autoscript_client import FIBSEM
import logging

PATH = "/home/patrick/development/meteor-fibsem-sim.odm.yaml"

from odemis import model
from Pyro4.core import isasync

import yaml

with open(PATH, 'r') as f:
    config = yaml.safe_load(f)

# from pprint import pprint
print(config["FIBSEM"])

logging.basicConfig(level=logging.DEBUG)
microscope = FIBSEM(name="fibsem", role="microscope", children=[], 
                    address=config["FIBSEM"]["init"]["address"])

print(f"Software Version: {microscope._swVersion}")
print(f"Hardware Version: {microscope._hwVersion}")


# Dependencies issue:
# requires msgpack==0.5.6, msgpack-numpy==0.4.4

# or
# msgpack==1.0.3 msgpack-numpy==0.4.8


In [ ]:
stage_position = microscope.get_stage_position()

print(f"Stage Position: {stage_position}")


In [ ]:
# imaging
microscope.set_dwell_time(1e-6, "electron")
microscope.set_field_of_view(150e-6, "electron")

microscope.run_auto_contrast_brightness("electron")
eb_image = microscope.acquire_image("electron")

microscope.set_dwell_time(1e-6, "ion")
microscope.set_field_of_view(150e-6, "ion")
microscope.run_auto_contrast_brightness("ion")
ib_image = microscope.acquire_image('ion')


import matplotlib.pyplot as plt
fig,ax = plt.subplots(1, 2, figsize=(15, 7))

ax[0].imshow(eb_image, cmap="gray")
ax[0].set_title(f"Electron Image")
ax[1].imshow(ib_image, cmap="gray")
ax[1].set_title(f"Ion Image")
plt.show()


In [ ]:
# live imaging
import time

channel = "electron"
imaging_state = microscope.get_imaging_state(channel)
logging.info(f"Imaging State: {imaging_state}")

microscope.start_acquisition(channel)

for i in range(5):
    imaging_state = microscope.get_imaging_state(channel)
    logging.info(f"Imaging State: {imaging_state}")
    
    image = microscope.get_last_image(
        channel=channel, 
        wait_for_frame=True
        )

    timestamp = time.time()
    plt.title(f"Image {i:02d} - {channel} - {timestamp}")
    plt.imshow(image, cmap="gray")
    plt.show()    

    time.sleep(1)

microscope.stop_acquisition(channel)

imaging_state = microscope.get_imaging_state(channel)
logging.info(f"Imaging State: {imaging_state}")

In [ ]:
# milling
microscope.clear_patterns()
import time

# define pattern
pattern_dict = {
    "center_x": 0,
    "center_y": 0,
    "width": 10e-6,
    "height": 10e-6,
    "depth": 0.5e-6
}

microscope.set_active_view(2)
microscope.set_active_device(2)
microscope.set_beam_current(2e-9, "ion")

# draw pattern
ret: dict = microscope.create_rectangle(pattern_dict) # 

# run milling
microscope.start_milling()

print(f"Estimated Time: {microscope.estimate_milling_time()} seconds")

# wait for milling to finish
while microscope.get_patterning_state() == "Running":
    print("Milling in progress...")
    time.sleep(1)

microscope.set_beam_current(41e-12, "ion")
microscope.clear_patterns()

print("Milling finished")

print(ret)

In [ ]:
microscope.set_beam_current(41e-12, "ion")


## Stage Movement


In [ ]:
stage_position = microscope.get_stage_position()
print(f"Stage Position", stage_position)

microscope.move_stage_relative({"x": 5e-6})

stage_position = microscope.get_stage_position()
print(f"Stage Position", stage_position)



In [ ]:
# beam info
channels = ["electron", "ion"]

for channel in channels:

    is_installed = microscope.beam_is_installed(channel)
    is_on = microscope.beam_is_on(channel)
    is_blanked = microscope.beam_is_blanked(channel)
    logging.info(f"Beam {channel} is Installed: {is_installed}")
    logging.info(f"Beam {channel} is On: {is_on}")
    logging.info(f"Beam {channel} is Blanked: {is_blanked}")

    scan_mode = microscope.get_scan_mode(channel)
    scan_mode_info = microscope.scan_mode_info(channel)
    logging.info(f"Scan Mode {channel}: {scan_mode}")

    beam_current = microscope.get_beam_current(channel)
    beam_current_info = microscope.beam_current_info(channel)
    logging.info(f"Beam Current {channel}: {beam_current} {beam_current_info}")

    voltage = microscope.get_high_voltage(channel)
    voltage_info = microscope.high_voltage_info(channel)
    logging.info(f"Voltage {channel}: {voltage} {voltage_info}")

    scanning_size = microscope.get_field_of_view(channel)
    scanning_size_info = microscope.field_of_view_info(channel)
    logging.info(f"Scanning Size {channel}: {scanning_size} {scanning_size_info}")

    if channel == "electron":
        spot_size = microscope.get_spotsize(channel)
        spot_size_info = microscope.spotsize_info(channel)
        logging.info(f"Spot Size {channel}: {spot_size}")
        logging.info(f"Spot Size Info {channel}: {spot_size_info}")

    working_distance = microscope.get_working_distance(channel)
    working_distance_info = microscope.working_distance_info(channel)
    logging.info(f"Working Distance {channel}: {working_distance} {working_distance_info}")

    stigmator = microscope.get_stigmator(channel)
    stigmator_info = microscope.stigmator_info(channel)
    logging.info(f"Stigmator {channel}: {stigmator} {stigmator_info}")

    dwell_time = microscope.get_dwell_time(channel)
    dwell_time_info = microscope.dwell_time_info(channel)
    logging.info(f"Dwell Time {channel}: {dwell_time} {dwell_time_info}")

    beam_shift = microscope.get_beam_shift(channel)
    beam_shift_info = microscope.beam_shift_info(channel)
    logging.info(f"Beam Shift {channel}: {beam_shift} {beam_shift_info}")

    resolution = microscope.get_resolution(channel)
    resolution_info = microscope.resolution_info(channel)
    logging.info(f"Resolution {channel}: {resolution} {resolution_info}")

    scan_rotation = microscope.get_scan_rotation(channel)
    scan_rotation_info = microscope.scan_rotation_info(channel)
    logging.info(f"Scan Rotation {channel}: {scan_rotation} {scan_rotation_info}")

    # Detector
    detector_mode = microscope.get_detector_mode(channel)
    detector_mode_info = microscope.detector_mode_info(channel)
    logging.info(f"Detector Mode {channel}: {detector_mode} {detector_mode_info}")

    detector_type = microscope.get_detector_type(channel)
    detector_type_info = microscope.detector_type_info(channel)
    logging.info(f"Detector Type {channel}: {detector_type} {detector_type_info}")

    detector_brightness = microscope.get_brightness(channel)
    detector_contrast = microscope.get_contrast(channel)
    logging.info(f"Detector Brightness {channel}: {detector_brightness}")
    logging.info(f"Detector Contrast {channel}: {detector_contrast}")

In [ ]:
# set api

for channel in channels:
    microscope.set_high_voltage(30e3, channel)
    microscope.set_beam_current(1e-9, channel)
    microscope.set_spotsize(3, channel)
    microscope.set_field_of_view(150e-6, channel)
    microscope.set_working_distance(10e-3, channel)
    microscope.set_stigmator(0, 0, channel)
    microscope.set_dwell_time(1e-6, channel)
    microscope.set_beam_shift(0, 0, channel)
    microscope.set_resolution([1024, 1024], channel)
    microscope.set_scan_rotation(0, channel)
    microscope.set_detector_mode("BackscatterElectrons", channel)
    microscope.set_detector_type("TLD", channel)
    microscope.set_brightness(0.5, channel)
    microscope.set_contrast(0.5, channel)

    # get the same values, and log them
    voltage = microscope.get_high_voltage(channel)
    voltage_info = microscope.high_voltage_info(channel)
    logging.info(f"Voltage {channel}: {voltage} {voltage_info}")

    beam_current = microscope.get_beam_current(channel)
    beam_current_info = microscope.beam_current_info(channel)
    logging.info(f"Beam Current {channel}: {beam_current} {beam_current_info}")

    spot_size = microscope.get_spotsize(channel)
    spot_size_info = microscope.spotsize_info(channel)
    logging.info(f"Spot Size {channel}: {spot_size}")

    field_of_view = microscope.get_field_of_view(channel)
    field_of_view_info = microscope.field_of_view_info(channel)
    logging.info(f"Field of View {channel}: {field_of_view} {field_of_view_info}")

    working_distance = microscope.get_working_distance(channel)
    working_distance_info = microscope.working_distance_info(channel)
    logging.info(f"Working Distance {channel}: {working_distance} {working_distance_info}")

    stigmator = microscope.get_stigmator(channel)
    stigmator_info = microscope.stigmator_info(channel)
    logging.info(f"Stigmator {channel}: {stigmator} {stigmator_info}")

    dwell_time = microscope.get_dwell_time(channel)
    dwell_time_info = microscope.dwell_time_info(channel)
    logging.info(f"Dwell Time {channel}: {dwell_time} {dwell_time_info}")

    beam_shift = microscope.get_beam_shift(channel)
    beam_shift_info = microscope.beam_shift_info(channel)
    logging.info(f"Beam Shift {channel}: {beam_shift} {beam_shift_info}")

    resolution = microscope.get_resolution(channel)
    resolution_info = microscope.resolution_info(channel)
    logging.info(f"Resolution {channel}: {resolution} {resolution_info}")
    
    scan_rotation = microscope.get_scan_rotation(channel)
    scan_rotation_info = microscope.scan_rotation_info(channel)
    logging.info(f"Scan Rotation {channel}: {scan_rotation} {scan_rotation_info}")

    detector_mode = microscope.get_detector_mode(channel)
    detector_mode_info = microscope.detector_mode_info(channel)
    logging.info(f"Detector Mode {channel}: {detector_mode} {detector_mode_info}")

    detector_type = microscope.get_detector_type(channel)
    detector_type_info = microscope.detector_type_info(channel)
    logging.info(f"Detector Type {channel}: {detector_type} {detector_type_info}")

    detector_brightness = microscope.get_brightness(channel)
    detector_contrast = microscope.get_contrast(channel)
    logging.info(f"Detector Brightness {channel}: {detector_brightness}")
    logging.info(f"Detector Contrast {channel}: {detector_contrast}")

# set api



In [ ]:
beam_shift_info = microscope.beam_shift_info("electron")

print(beam_shift_info["range"]["x"][0])

## Components

In [ ]:
# list components

%load_ext autoreload
%autoreload 2


from odemis import model
from pprint import pprint
import math
import json 

components = model.getComponents()

for c in components:
    print(c, c.role, c.getMetadata())
    # if "Linked YZ" in c.name:a
    #     print(c, c.role, c.getMetadata())

    #     ddict = c.getMetadata()
    #     pretilt= ddict[model.MD_ROTATION_COR]
    #     print(math.degrees(pretilt))

print(model.getMicroscope())

In [ ]:
fibsem = model.getComponent(role="fibsem")
print(fibsem.get_stage_position())

eb_image = fibsem.acquire_image("electron")
ib_image = fibsem.acquire_image("ion")

print("Electron Image: ", eb_image.shape)
print("Ion Image: ", ib_image.shape)

In [ ]:
%load_ext autoreload
%autoreload 2


from odemis import model

In [ ]:
stage = model.getComponent(role="stage-bare")

print(stage._getPosition())

stage.moveRelSync({"x": 50e-3})

import time
time.sleep(2)

print(stage._getPosition())

stage.moveRelSync({"y": 50e-3})

print(stage._getPosition())

import time
time.sleep(2)

stage.moveAbsSync({"x": 0, "y": 0, "z": 0})

print(stage._getPosition())


In [ ]:
stage.moveAbsSync({"x": 0, "y": 0, "z": 0, "rx": 0, "rz": 0})
print(stage._getPosition())

In [ ]:
electron_beam = model.getComponent(role="e-beam")
electron_beam.beamCurrent.value = 2e-9

In [ ]:
print(electron_beam.beamCurrent.value)

In [ ]:
%matplotlib inline

from odemis import model
from odemis.acq.stream import SEMStream, FIBStream
from odemis.acq.acqmng import acquire
import matplotlib.pyplot as plt

# setup electron beam, det
electron_beam = model.getComponent(role="e-beam")
electron_det = model.getComponent(role="electron-detector")

# setup ion beam, det
ion_beam = model.getComponent(role="ion-beam")
ion_det = model.getComponent(role="ion-detector")

# create streams
sem_stream = SEMStream("sem-stream", electron_det, electron_det.data, electron_beam)
fib_stream = FIBStream("fib-stream", ion_det, ion_det.data, ion_beam)


In [ ]:
%matplotlib inline

# set imaging settings
ion_beam.dwellTime.value = 5e-6
ion_beam.horizontalFoV.value = 400e-6

# acquire streams
f = acquire([sem_stream, fib_stream])
data, ex = f.result(timeout=600)
sem_image, fib_image = data # unpack data

# plot images
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
plt.show()

print(sem_stream.raw[0].metadata)
print(fib_stream.raw[0].metadata)

from odemis import dataio
fn = "electron_image.ome.tiff"
exporter = dataio.find_fittest_converter(fn)
exporter.export(fn, sem_image)

fn = "ion_image.ome.tiff"
exporter = dataio.find_fittest_converter(fn)
exporter.export(fn, fib_image)

In [ ]:
# save position
initial_stage_position = stage.position.value

In [ ]:
from odemis import model
stage = model.getComponent(role="stage-bare")


# move stage by 50e-6
f = stage.moveRel({"x": 50e-6, "y": 50e-6})
f.result()

print(initial_stage_position)
print(stage.position.value)


In [ ]:
print(initial_stage_position)
print(stage.position.value)

initial_stage_position.pop("coordinate_system")
f = stage.moveAbs(initial_stage_position)
f.result()

print(initial_stage_position)
print(stage.position.value)

In [ ]:
import time


microscope = model.getComponent(role="fibsem")
microscope.clear_patterns()

# acquire streams
f = acquire([sem_stream, fib_stream])
data, ex = f.result(timeout=600)

# plot images
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
plt.show()

microscope.create_rectangle({"center_x": 0, "center_y": 0, 
                         "width": 10e-6, "height": 10e-6, 
                         "depth": 0.5e-6})
microscope.set_beam_current(2e-9, "ion")
microscope.start_milling()

while microscope.get_patterning_state() == "Running":
    print("Milling in progress...")
    time.sleep(1)

microscope.set_beam_current(40e-12, "ion")
microscope.clear_patterns()

# acquire streams
f = acquire([sem_stream, fib_stream])
data, ex = f.result(timeout=600)

# plot images
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].imshow(sem_image, cmap="gray")
ax[1].imshow(fib_image, cmap="gray")
plt.show()

In [ ]:
from odemis import model
stage = model.getComponent(role="stage-bare")

print(stage.position.value)

stage.moveAbsSync({"x": 0, "y": 0, "z": 0})